In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [19]:
#simple CNN network
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels = 8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        return x

In [20]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> saving checkpoint")
    torch.save(state, filename)

In [28]:
def load_checkpoint(checkpoint):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [29]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
#hyperparameters
in_channels = 1 #MNIST
num_classes = 10 #MNIST
learning_rate = 1e-4
batch_size = 1024
num_epochs = 10
load_model = True

In [31]:
#load data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [32]:
#initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

In [33]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading checkpoint


In [36]:
#train network
for epoch in range(num_epochs):
    losses = []
    
    if epoch % 3 == 0:
        checkpoint = {'state_dict' : model.state_dict(), 'optimizer' : optimizer.state_dict()}
        save_checkpoint(checkpoint)    
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        #get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or adam step
        optimizer.step()
        
    mean_loss = sum(losses)/len(losses)
    print(f'loss_at epoch : {epoch} was {mean_loss:5f}')

=> saving checkpoint
loss_at epoch : 0 was 0.368889
loss_at epoch : 1 was 0.351384
loss_at epoch : 2 was 0.336931
=> saving checkpoint
loss_at epoch : 3 was 0.323272
loss_at epoch : 4 was 0.312469
loss_at epoch : 5 was 0.300439
=> saving checkpoint
loss_at epoch : 6 was 0.291160
loss_at epoch : 7 was 0.282817
loss_at epoch : 8 was 0.273735
=> saving checkpoint
loss_at epoch : 9 was 0.266075
